# Инференс и валидация

Теперь, когда у вас есть обученная сеть, вы можете использовать её для прогнозирования. Это обычно называется **инференсом** (inference). Однако нейронные сети имеют тенденцию показывать *слишком хорошие* результаты на обучающих данных и не могут обобщать свою работу на данные, которые не были доступны при обучении. Это называется **переобучением**, и оно ухудшает качество инференса. Чтобы проверить наличие переобучения в процессе обучения, мы измеряем метрики на данных, не входящих в обучающий набор, которые называют **валидационным** набором. Мы избегаем переобучения с помощью регуляризации, такой как дроп-аут, в то время как следим за метриками на валидационной выборке в процессе обучения. В этом блокноте покажем, как это сделать в PyTorch. 

Как обычно, давайте начнём с загрузки набора данных через torchvision. Вы узнаете больше о torchvision и загрузке данных в следующей части. На этот раз мы воспользуемся тестовым набором, который вы можете получить, установив `train=False`:

```python
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
```

Тестовый набор содержит изображения, точно так же, как и тренировочный набор. Обычно вы увидите, что 10-20% оригинального набора данных отложены для тестирования и валидации, а оставшаяся часть используется для обучения.

In [ ]:
import torch
from torchvision import datasets, transforms

# Определим трансформацию для нормализации данных
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Загрузим тренировочные данные
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Загрузим тестовые данные
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

Создадим модель нейронной сети.

In [ ]:
from torch import nn, optim
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        # убедимся, что входной тензор развернут в вектор-строку (flattened)
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

Цель валидации состоит в том, чтобы измерить метрики модели на данных, которые не входят в тренировочный набор. Выбор метрик зависит во многом от разработчика. Обычно это просто точность (accuracy), процент классов, которые сеть предсказывает правильно. Другие варианты включают [precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall#Definition_(classification_context)) и top-5 error rate. Здесь мы сосредоточимся на точности. Сначала сделаем прямой проход с одним батчем из тестового набора.

In [ ]:
model = Classifier()

images, labels = next(iter(testloader))
# Получим вероятности классов
ps = torch.exp(model(images))
# Убедимся, что форма правильная, мы должны получить 10 вероятностей классов для 64 примеров
print(ps.shape)

С вероятностями мы можем получить наиболее вероятный класс, используя метод `ps.topk`. Это возвращает $k$ наивысших значений. Поскольку мы просто хотим наиболее вероятный класс, мы можем использовать `ps.topk(1)`. Это возвращает кортеж из наивысших значений и соответствующих индексов. Если наивысшее значение является пятым элементом, мы получим 4 как индекс.

In [ ]:
top_p, top_class = ps.topk(1, dim=1)
# Посмотрим на наиболее вероятные классы для первых 10 примеров
print(top_class[:10,:])

Теперь мы можем проверить, совпадают ли предсказанные классы с метками. Это легко сделать, сравнив `top_class` и `labels`, но мы должны быть осторожны с формами. Здесь `top_class` — это 2D тензор с формой `(64, 1)`, а `labels` — это 1D с формой `(64)`. Чтобы получить нужное равенство, `top_class` и `labels` должны иметь одинаковую форму.

Если мы сделаем

```python
equals = top_class == labels
```

`equals` будет иметь форму `(64, 64)`. Происходит сравнение одного элемента в каждой строке `top_class` с каждым элементом в `labels`, что возвращает 64 значения True/False для каждой строки.

In [ ]:
equals = top_class == labels.view(*top_class.shape)

Теперь нам нужно рассчитать процент правильных предсказаний. `equals` имеет двоичные значения, либо 0, либо 1. Это означает, что если мы просто суммируем все значения и делим на количество значений, мы получим процент правильных предсказаний. Это та же операция, что и вычисление среднего, поэтому мы можем получить точность с помощью вызова `torch.mean`. Если бы все было так просто. Если вы попробуете `torch.mean(equals)`, вы получите ошибку

```
RuntimeError: mean is not implemented for type torch.ByteTensor
```

Это происходит потому, что `equals` имеет тип `torch.ByteTensor`, но для этого типа `torch.mean` не реализован. Поэтому нам нужно преобразовать `equals` в float тензор с плавающей запятой. Обратите внимание, что когда мы вызываем `torch.mean`, это возвращает скалярный тензор. Чтобы получить фактическое значение как float, нам нужно будет использовать `accuracy.item()`.

In [ ]:
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Точность: {accuracy.item()*100}%')

Сеть не обучена, поэтому она делает случайные предположения, и мы должны увидеть точность около 10%. Теперь давайте обучим нашу сеть и включим шаг валидации, чтобы измерить, насколько хорошо сеть работает на тестовом наборе. Поскольку мы не обновляем параметры в цикле валидации, мы можем ускорить процесс, отключив градиенты с помощью `torch.no_grad()`:

```python
# отключение градиентов
with torch.no_grad():
    # валидация
    for images, labels in testloader:
        ...
```

>**Упражнение:** Реализуйте обучение сети совместно с циклом валидации.

In [ ]:
## TODO

model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 30
steps = 0

train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        
        optimizer.zero_grad()
        
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        ## TODO: Реализуйте цикл валидации и вывод точности на валидационной выборке
        print(f'Accuracy: {accuracy.item()*100}%')

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)
plt.show()

## Переобучение (overfitting)

Если мы посмотрим на функцию потерь при обучении и валидации по мере обучения сети, мы можем увидеть явление, известное как переобучение (overfitting).

<img src='assets/overfitting.png' width=450px>

Сеть учит тренировочный набор всё лучше и лучше, что приводит к снижению потерь при обучении. Однако она начинает испытывать проблемы с обобщением данных, выходящих за пределы тренировочного набора, что приводит к росту потерь на валидации. Конечная цель любой модели глубокого обучения — делать предсказания на новых данных, поэтому мы должны стремиться получить как можно более низкую потерю на валидации. Один из вариантов — использовать версию модели с наименьшей потерей на валидации, здесь это версия примерно через 8-10 эпох обучения. Эта стратегия называется *ранней остановкой* (early-stopping). На практике вы будете сохранять модель регулярно, пока обучаете её, а затем позже выбирать модель с наименьшей потерей на валидации.

Наиболее распространённым методом снижения переобучения (помимо ранней остановки) является *дроп-аут* (dropout), когда мы случайным образом исключаем узлы. Это заставляет сеть делиться информацией между весами, увеличивая её способность обобщать работу на новые данные. Добавить дроп-аут в PyTorch очень просто, используя модуль [`nn.Dropout`](https://pytorch.org/docs/stable/nn.html#torch.nn.Dropout).

```python
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
        # Модуль дропаута с вероятностью исключения 0.2
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        # убедимся, что входной тензор развернут (flattened)
        x = x.view(x.shape[0], -1)
        
        # Теперь с дроп-аутом
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        
        # вывод, поэтому без дропаута
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x
```

Во время обучения мы хотим использовать дроп-аут, чтобы предотвратить переобучение, но во время инференса мы хотим использовать всю сеть. Поэтому нам нужно отключить дроп-аут во время валидации, тестирования и когда мы используем сеть для предсказаний. Для этого используйте `model.eval()`. Это устанавливает модель в режим валидации (evaluation mode), где вероятность дропаута равна 0. Вы можете снова включить дроп-аут, установив модель в режим обучения с помощью `model.train()`. В общем, структура цикла валидации будет выглядеть следующим образом: вы отключаете градиенты, устанавливаете модель в режим валидации, вычисляете функцию потерь и метрики валидации, а затем возвращаете модель в режим обучения.

```python
# отключение градиентов
with torch.no_grad():
    
    # установить модель в режим оценки
    model.eval()
    
    # валидационный проход здесь
    for images, labels in testloader:
        ...

# вернуть модель в режим обучения
model.train()
```

> **Упражнение:** Добавьте дроп-аут в вашу модель и обучите её снова на Fashion-MNIST. Посмотрите, сможете ли вы получить более низкую потерю на валидации.

In [ ]:
## TODO: Определите модель с добавленным дроп-аутом


In [ ]:
## TODO: Обучите модель с дроп-аутом, добавьте мониторинг обучения с расчетом функции потерь и точности на валидационной выборке


In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)
plt.show()

## Инференс (inference)

Теперь, когда модель обучена, мы можем использовать её для инференса. Мы уже делали это раньше, но теперь нам нужно помнить о установке модели в режим валидации с `model.eval()`. Также можно отключить autograd с помощью менеджера контекста `torch.no_grad()`.

In [ ]:
# Импортируем вспомогательный модуль (должен находиться в репозитории)
import helper

# Примените свою сеть.

model.eval()

dataiter = iter(testloader)
images, labels = next(dataiter)
img = images[0]
# Преобразуем 2D изображение в 1D вектор
img = img.view(1, 784)

# Рассчитаем вероятности классов (softmax) для img
with torch.no_grad():
    output = model.forward(img)

ps = torch.exp(output)

# Построим изображение и вероятности
helper.view_classify(img.view(1, 28, 28), ps, version='Fashion')

В следующей части будет продемонстрировано, как сохранить ваши обученные модели. Обычно вы не хотите обучать модель каждый раз, когда она вам нужна. Вместо этого вы будете обучать один раз, сохранять её, а затем загружать модель, когда захотите обучить подольше или использовать её для инференса.